# Practicumopdrachten Week 2.3 (eerste kans)

## Dependencies importeren

Kopieer in het codeblok hieronder van het vorige practicum de import-code voor de dependencies die het vaakst worden gebruikt om data in te lezen. Geef er ook de gebruikelijke aliassen aan.<br>
Zet eventuele warnings uit.

In [ ]:

import warnings
warnings.simplefilter('ignore')
import pandas as pd
import sqlite3
import numpy as np

Zet het bestand go_sales.sqlite in hetzelfde mapje als dit notebook.

## Databasetabellen inlezen

Kopieer in het codeblok hieronder van het vorige practicum de code om een connectie met het bestand go_sales.sqlite te maken.

In [ ]:
go_sales_con = sqlite3.connect('Great_Outdoors_Data_SQLite\go_sales.sqlite')

Lees van de ingelezen go_sales-database te volgende tabellen in met behulp van "SELECT * FROM *tabel*".
- product
- product_type
- product_line
- sales_staff
- sales_branch
- retailer_site
- country
- order_header
- order_details
- target
- returned_item
- return_reason

In [ ]:
queries = {
    'product': 'SELECT * FROM product',
    'product_type': 'SELECT * FROM product_type',
    'product_line': 'SELECT * FROM product_line',
    'sales_staff': 'SELECT * FROM sales_staff',
    'sales_branch': 'SELECT * FROM sales_branch',
    'retailer_site': 'SELECT * FROM retailer_site',
    'country': 'SELECT * FROM country',
    'order_header': 'SELECT * FROM order_header',
    'order_details': 'SELECT * FROM order_details',
    'returned_item': 'SELECT * FROM returned_item',
    'return_reason': 'SELECT * FROM return_reason'
}


dataframes = {}

# Lees elke tabel in een DataFrame
for table_name, query in queries.items():
    dataframes[table_name] = pd.read_sql_query(query, go_sales_con)

#als je ik elk tabel als een dataframe/ variabele wil behandelen of aanroepen moet ik dit uitvoeren.
for table_name, df in dataframes.items():
    globals()[table_name] = df
    
dataframes

Let op! Voor alle onderstaande opdrachten mag je <u>alleen Python</u> gebruiken, <u>geen SQL!</u>

## Selecties op één tabel zonder functies

Geef een overzicht met daarin de producten en hun productiekosten waarvan de productiekosten lager dan 100 dollar en hoger dan 50 dollar ligt. (2 kolommen, 23 rijen)

In [ ]:
# maak productie kost float omdat het nu als string gezien word.
product['PRODUCTION_COST'] = product['PRODUCTION_COST'].astype(float)

Productie_kost = (product['PRODUCTION_COST'] < 100) & (product['PRODUCTION_COST'] > 50)

resultaat = product.loc[(Productie_kost), ['PRODUCT_NAME', 'PRODUCTION_COST']]
resultaat

Geef een overzicht met daarin de producten en hun marge waarvan de marge lager dan 20 % of hoger dan 60 % ligt. (2 kolommen, 7 rijen) 

In [ ]:
#een margin wordt eerst als object gezien, dus deze moet eerst omgezet worden naar float

product['MARGIN'] = product['MARGIN'].astype(float)
Marge = (product['MARGIN'] < .20) | (product['MARGIN'] > .60)

resultaat = product.loc[(Marge), ['PRODUCT_NAME', 'MARGIN']]
resultaat

Geef een overzicht met daarin de landen waar met francs wordt betaald. Sorteer de uitkomst op land.  (1 kolom, 3 rijen)

In [ ]:
francs = country['CURRENCY_NAME'] == 'francs'

resultaat = country.loc[(francs), ['COUNTRY']].sort_values(by = 'COUNTRY')
resultaat
#sorteren doe je door daarna .sort_values te doen.

Geef een overzicht met daarin de verschillende introductiedatums waarop producten met meer dan 50% marge worden geïntroduceerd (1 kolom, 7 rijen) 

In [ ]:
product['MARGIN'] = product['MARGIN'].astype(float)
Datums = product['MARGIN'] > .50

resultaat = product.loc[(Datums), ['INTRODUCTION_DATE']].drop_duplicates('INTRODUCTION_DATE')   
#bestaande code geeft veel duplicates dus word drop_duplicates aangeroepen
resultaat

Geef een overzicht met daarin het eerste adres en de stad van verkoopafdelingen waarvan zowel het tweede adres als de regio bekend is (2 kolommen, 7 rijen)

In [ ]:
Adress = (sales_branch['ADDRESS2'].notna()) & (sales_branch['REGION'].notna())
resultaat = sales_branch.loc[(Adress), ['ADDRESS1', 'CITY']]
resultaat

Geef een overzicht met daarin de landen waar met dollars (dollars of new dollar) wordt betaald. Sorteer de uitkomst op land. (1 kolom, 4 rijen) 

In [ ]:
Dollar = (country['CURRENCY_NAME'] == 'dollars') | (country['CURRENCY_NAME'] == 'new dollar')
resultaat = country.loc[(Dollar), ['COUNTRY']].sort_values(by = 'COUNTRY')
resultaat

Geef een overzicht met daarin beide adressen en de stad van vestigingen van klanten waarvan de postcode begint met een ‘D’ (van duitsland). Filter op vestigingen die een tweede adres hebben. (3 kolommen, 2 rijen) 

In [ ]:
Vestiging = (retailer_site['POSTAL_ZONE'].str[0] == 'D')

resultaat = retailer_site.loc[(Vestiging) & retailer_site['ADDRESS2'].notna(), ['ADDRESS1', 'ADDRESS2', 'CITY']]
resultaat

## Selecties op één tabel met functies

Geef het totaal aantal producten dat is teruggebracht (1 waarde) 

In [ ]:
returned_item['RETURN_QUANTITY'] = returned_item['RETURN_QUANTITY'].astype(float)
totaal = returned_item['RETURN_QUANTITY'].sum()
print(totaal)

#14306.0

Geef het aantal regio’s waarin verkoopafdelingen gevestigd zijn. (1 waarde)

In [ ]:
Regio = sales_branch['REGION'].drop_duplicates().count()
print(Regio)

#Beide kunnen gebruikt worden onder meer "Geschikt"

aantal_regios = sales_branch['REGION'].nunique()
print(aantal_regios)




Maak 3 variabelen:
- Een met de laagste
- Een met de hoogste
- Een met de gemiddelde (afgerond op 2 decimalen)

marge van producten (3 kolommen, 1 rij) 

In [ ]:
laagsteMargin = product['MARGIN'].min()
hoogsteMargin = product['MARGIN'].max()
gemMargin = product['MARGIN'].mean()

df = {
    'laagste' : [laagsteMargin],
    'hoogste' : [hoogsteMargin],
    'gem' : [gemMargin]
}

Resultaat = pd.DataFrame(df)
Resultaat

#onderstaande regel kan ook .to_fram om om te zetten naar dataframe en .T om te transponeren (rijen worden kolomen, kolomen worden rijen)
resultaat = product['MARGIN'].agg(['min', 'max', 'mean']).to_frame().T
resultaat


Geef het aantal vestigingen van klanten waarvan het 2e adres niet bekend is (1 waarde)

In [ ]:
aantal = retailer_site['ADDRESS2'].isna().sum()
print(aantal)

#286

Geef de gemiddelde kostprijs van de verkochte producten waarop korting (unit_sale_price < unit_price) is verleend (1 waarde) 

In [ ]:

order_details['UNIT_SALE_PRICE'] = order_details['UNIT_SALE_PRICE'].astype(float)
order_details['UNIT_PRICE'] = order_details['UNIT_PRICE'].astype(float)
order_details['UNIT_COST'] = order_details['UNIT_COST'].astype(float)


#de loc is de functie .mean ook ?

gem = order_details.loc[order_details['UNIT_SALE_PRICE'] < order_details['UNIT_PRICE'], 'UNIT_COST'].mean()
#gem = (order_details[order_details['UNIT_SALE_PRICE'] < order_details['UNIT_PRICE']]['UNIT_COST']).mean()

print(gem)
#125.42305809556548


Geef een overzicht met daarin het aantal medewerkers per medewerkersfunctie (2 kolommen, 7 rijen) 

In [ ]:
sales_staff.groupby('POSITION_EN', as_index = False)['SALES_STAFF_CODE'].count().rename(columns = {'POSITION_EN' : 'FUNCTIE', 'SALES_STAFF_CODE' : 'AANTAL'})

Geef een overzicht met daarin per telefoonnummer het aantal medewerkers dat op dat telefoonnummer bereikbaar is. Toon alleen de telefoonnummer waarop meer dan 4 medewerkers bereikbaar zijn. (2 kolommen, 10 rijen) 

In [29]:

resultaat = sales_staff.groupby('WORK_PHONE', as_index=False)['SALES_STAFF_CODE'].count()

#zo of met .query achter.count
meer_dan_4 = resultaat[resultaat['SALES_STAFF_CODE'] > 4]

print(meer_dan_4)

             WORK_PHONE  SALES_STAFF_CODE
4     +(44) 121 3505267                 5
10  +(61) 03 2982  4242                 5
11  +31 (0)20 692 93 94                 5
13    +33 1 68 94 52 20                 5
18     1 (206) 292-0012                 5
19     1 (305) 557-4810                 6
20     1 (310) 281-5722                 5
21     1 (403) 232-5986                 5
22     1 (416) 493-5595                 5
23     1 (617) 268-6754                 6


## Selecties op meerdere tabellen zonder functies

Geef een overzicht met daarin het eerste adres en de stad van vestigingen van klanten uit ‘Netherlands’ (2 kolommen, 20 rijen) 

Geef een overzicht met daarin de productnamen die tot het producttype ‘Eyewear’ behoren. (1 kolom, 5 rijen) 

Geef een overzicht met daarin alle unieke eerste adressen van klantvestigingen en de voornaam en achternaam van de verkopers die ‘Branch Manager’ zijn en aan deze vestigingen hebben verkocht (3 kolommen, 1 rij) 

Geef een overzicht met daarin van de verkopers hun functie en indien zij iets hebben verkocht de datum waarop de verkoop heeft plaatsgevonden. Laat alleen de verschillende namen van de posities zien van de verkopers die het woord ‘Manager’ in hun positienaam hebben staan. (2 kolommen, 9 rijen) 

Geef een overzicht met daarin de verschillende namen van producten en bijbehorende namen van producttypen van de producten waarvoor ooit meer dan 750 stuks tegelijk verkocht zijn. (2 kolommen, 9 rijen) 

Geef een overzicht met daarin de productnamen waarvan ooit meer dan 40% korting is verleend. De formule voor korting is: (unit_price - unit_sale_price) / unit_price (1 kolom, 8 rijen) 

Geef een overzicht met daarin de retourreden van producten waarvan ooit meer dan 90% van de aangeschafte hoeveelheid is teruggebracht (return_quantity/quantity). (1 kolom, 3 rijen) 

## Selecties op meerdere tabellen met functies

Geef een overzicht met daarin per producttype het aantal producten die tot dat producttype behoren. (2 kolommen, 21 rijen) 

Geef een overzicht met daarin per land het aantal vestigingen van klanten die zich in dat land bevinden. (2 kolommen, 21 rijen) 

Geef een overzicht met daarin van de producten behorend tot het producttype ‘Cooking Gear’ per productnaam de totaal verkochte hoeveelheid en de gemiddelde verkoopprijs. Sorteer de uitkomst op totaal verkochte hoeveelheid. (4 kolommen, 10 rijen) 

Geef een overzicht met daarin van de verkopers behorend tot de verkoopafdeling uit ‘Amsterdam’ de voornaam en achternaam van de verkoper en zijn laagste en hoogste verkooptarget van 2006. (4 kolommen, 4 rijen) 

Geef een overzicht met daarin per land de naam van het land, de naam van de stad waar de verkoopafdeling is gevestigd (noem de kolomnaam in het overzicht ‘verkoper’) en het aantal steden waar zich klanten bevinden in dat land (noem de kolomnaam in het overzicht ‘klanten’) (3 kolommen, 29 rijen) 

## Pythonvertalingen van SUBSELECT en UNION met o.a. for-loops

Geef een overzicht met daarin de voornaam en de achternaam van de medewerkers die nog nooit wat hebben verkocht (2 kolommen, 25 rijen) 

Geef een overzicht met daarin het aantal producten waarvan de marge lager is dan de gemiddelde marge van alle producten samen. Geef in het overzicht tevens aan wat de gemiddelde marge is van dit aantal producten waarvan de marge lager dan de gemiddelde marge van alle producten samen is. (1 kolom, 2 rijen) 

Geef een overzicht met daarin de namen van de producten die voor meer dan 500 (verkoopprijs) zijn verkocht maar nooit zijn teruggebracht. (1 kolom, 13 rijen) 

Geef een overzicht met daarin per (achternaam van) medewerker of hij/zij manager is of niet, door deze informatie toe te voegen als extra 'Ja/Nee'-kolom.<br>
Hint: gebruik een for-loop waarin je o.a. bepaalt of het woord 'Manager' in de functie (position_en) staat. (2 kolommen, 102 rijen).

Met de onderstaande code laat je Python het huidige jaar uitrekenen.

In [ ]:
from datetime import date
date.today().year

Met de onderstaande code selecteer je op een bepaald jaartal uit een datum.

In [ ]:
from datetime import datetime

date_str = '16-8-2013'
date_format = '%d-%m-%Y'
date_obj = datetime.strptime(date_str, date_format)

date_obj.year

Geef met behulp van bovenstaande hulpcode een overzicht met daarin op basis van het aantal jaar dat iemand in dienst is of een medewerker ‘kort in dienst’ (minder dan 25 jaar in dienst) of een ‘lang in dienst’ (groter gelijk dan 12 jaar in dienst) is. Geef daarbij per medewerker in een aparte kolom zowel ‘kort in dienst’ als ‘lang in dienst’ aan. Gebruik (wederom) een for-loop.<br>
(2 kolommen, 102 rijen) 